In [0]:
# https://keras.io/
!pip install -q keras
import keras

import scipy
import numpy as np
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.preprocessing.image import ImageDataGenerator

import h5py
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

## **Here we have implemented densnet model for cifar10 database using only 50 epochs**

We perform the learning task of densenet in two steps. First we take images of size 32X32 and downsize them to 16x16 and train the network for **20 epochs** defined by model **pre_model28_l** with parameters count **2,062,538**. 
Then we save these weights change the output layer of the previous model and make a new model **model28_l** with parameters count **2,072,618** and train it for actual images of size 32x32 for **30 epochs**. This sort of transfer-learning should facilitate the learning of network quite easier/faster.

**Key techniques used are:**
**1. Image Augmentation to avoid overfitting to the training data like **
        rotation_range=12,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images 
        
**2. learning-rate scheduling for damping the ocsillations in validation accuracy defined by** 
        lr_schedule (for pre_model28_l) and 
        lr_schedule_1(for model28_l)
        
 **Results of this experiments**
 Training accuracy: 93.91% and validation accuracy: 90.98%

# **Let us begin:**

**lr_schedule** for **pre_model28_l**

In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 15:
        lrate = 0.0001
         
    return lrate

**lr_schedule_1** for **model28_l**

In [0]:
def lr_schedule_1(epoch):
    if epoch > 26:
        lrate = 0.0001
    elif epoch > 23:
        lrate = 0.0002
    elif epoch > 18:
        lrate = 0.0003
    elif epoch > 13:
        lrate = 0.0005
    else:
        lrate = 0.001
         
    return lrate

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# **Hyperparameters**
**l=20, num_filter=32, compression=0.5 and drop_out=0.1**

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 20
num_filter = 32
compression = 0.5
dropout_rate = 0.1

**load cifar10 dataset**

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## **RESIZING IMAGES FOR PRE-TRAINING**

In [0]:
x_train.shape

(50000, 32, 32, 3)

In [0]:
x_train_resized = np.array([scipy.misc.imresize(image, (16, 16),mode='RGB') for image in x_train])
x_train_resized.shape

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


(50000, 16, 16, 3)

In [0]:
x_test.shape

(10000, 32, 32, 3)

In [0]:

x_test_resized = np.array([scipy.misc.imresize(image, (16, 16),mode='RGB') for image in x_test])
x_test_resized.shape

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


(10000, 16, 16, 3)

In [0]:
img_height, img_width, channel = x_train_resized.shape[1],x_train_resized.shape[2],x_train_resized.shape[3] ###used for pre-training the model

# **Define data augmentation using ImageDataGenerator facility provided by keras**

In [0]:
# With data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=12,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

## **Denseblock, add_transition, output layer for the densenet given to us**

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization(name='Batch'+str(block)+str(_))(temp)
        relu = Activation('relu',name='Activation'+str(block)+str(_))(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same',name='Conv'+str(block)+str(_))(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate,name='Dropout'+str(block)+str(_))(Conv2D_3_3)
        concat = Concatenate(axis=-1,name='Concat'+str(block)+str(_))([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization(name='Batch'+'_transition'+str(block))(input)
    relu = Activation('relu',name='Activation'+'_transition'+str(block))(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same',name='Conv'+'_transition'+str(block))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate,name='Dropout'+'_transition'+str(block))(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2),name='AvgPooling'+'_transition'+str(block))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization(name='Batch_output_layer')(input)
    relu = Activation('relu',name='Activation_output_layer')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2),name='AvgPooling_output_layer')(relu)
    flat = Flatten(name='Flattern_output_layer')(AvgPooling)
    output = Dense(num_classes, activation='softmax',name='Dense_output_layer')(flat)
    
    return output

# **Defining pre_model28_l for input size 16x16**

In [0]:
dropout_rate = 0.1
input = Input(shape=(img_height, img_width, channel,),name='input_layer')
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same',name='Conv2D_1st')(input)
block=1
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)
block=2
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)
block=3
Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)
block=4
Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
pre_model28_l = Model(inputs=[input], outputs=[output])
pre_model28_l.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 16, 16, 3)    0                                            
__________________________________________________________________________________________________
Conv2D_1st (Conv2D)             (None, 16, 16, 32)   864         input_layer[0][0]                
__________________________________________________________________________________________________
Batch10 (BatchNormalization)    (None, 16, 16, 32)   128         Conv2D_1st[0][0]                 
__________________________________________________________________________________________________
Activation10 (Activation)       (None, 16, 16, 32)   0           Batch10[0][0]                    
__________________________________________________________________________________________________
Conv10 (Co

__________________________________________________________________________________________________
Activation39 (Activation)       (None, 4, 4, 160)    0           Batch39[0][0]                    
__________________________________________________________________________________________________
Conv39 (Conv2D)                 (None, 4, 4, 16)     23040       Activation39[0][0]               
__________________________________________________________________________________________________
Dropout39 (Dropout)             (None, 4, 4, 16)     0           Conv39[0][0]                     
__________________________________________________________________________________________________
Concat39 (Concatenate)          (None, 4, 4, 176)    0           Concat38[0][0]                   
                                                                 Dropout39[0][0]                  
__________________________________________________________________________________________________
Batch310 (

**Observe the definition of lr_changes which is a keras.callbacks.LearningRateSchedular allowing to change learning_rate at the given epoch**

In [0]:
weight_checksaver = keras.callbacks.ModelCheckpoint('cifar10_weights.{epoch:02d}-{val_loss:.3f}.hdf5',
                                                  monitor='val_loss', verbose=1, save_weights_only=True, period=5, mode = 'auto')
lr_changes = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

#define optimizer
opt_adam = keras.optimizers.Adam(lr=0.001,decay=1e-8)

# determine Loss function using defined Optimizer
pre_model28_l.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['accuracy'])

**After compiling the model we start learning by using fit_generator facility of keras**

In [0]:
#Total 20 epochs with Increasing Batch size
#No. of epochs=  epcohs - initial_epochs

#batch_size  epcohs  initial_epochs  # epochs run
# 16         2          0             2        
# 32         5          1             3      
# 48         10         3             5
# 64         20         10            10
#---------------------------------------------------
#Total Epochs                 =    20



epochs=2 #epochs run=2
batch_size = 16
history1=pre_model28_l.fit_generator(datagen.flow(x_train_resized,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_resized)/batch_size,
                         epochs=epochs,
                         initial_epoch=0,
                         verbose=1,
                         callbacks=[weight_checksaver,lr_changes],
                         validation_data=(x_test_resized,y_test)
                        )

epochs=5 #epochs run=3
batch_size = 32
history2=pre_model28_l.fit_generator(datagen.flow(x_train_resized,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_resized)/batch_size,
                         epochs=epochs,
                         initial_epoch=2,
                         verbose=1,
                         callbacks=[weight_checksaver,lr_changes],
                         validation_data=(x_test_resized,y_test)
                        )

epochs=10 #epochs run=5
batch_size = 48
history3=pre_model28_l.fit_generator(datagen.flow(x_train_resized,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_resized)/batch_size,
                         epochs=epochs,
                         initial_epoch=5,
                         verbose=1,
                         callbacks=[weight_checksaver,lr_changes],
                         validation_data=(x_test_resized,y_test)
                        )

epochs=20 #epochs run=10
batch_size = 64
history4=pre_model28_l.fit_generator(datagen.flow(x_train_resized,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_resized)/batch_size,
                         epochs=epochs,
                         initial_epoch=10,
                         verbose=1,
                         callbacks=[weight_checksaver,lr_changes],
                         validation_data=(x_test_resized,y_test)
                        )

Epoch 1/2

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
 293/3125 [=>............................] - ETA: 17:13 - loss: 2.0831 - acc: 0.2191

3124/3125 [============================>.] - ETA: 0s - loss: 1.7876 - acc: 0.3387

3125/3125 [==============================] - 979s 313ms/step - loss: 1.7877 - acc: 0.3387 - val_loss: 1.6219 - val_acc: 0.4107
Epoch 2/2

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
 267/3125 [=>............................] - ETA: 14:00 - loss: 1.5730 - acc: 0.4232

3124/3125 [============================>.] - ETA: 0s - loss: 1.4979 - acc: 0.4569

3125/3125 [==============================] - 954s 305ms/step - loss: 1.4979 - acc: 0.4569 - val_loss: 1.3765 - val_acc: 0.5214
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
 268/1562 [====>.........................] - ETA: 6:53 - loss: 1.3023 - acc: 0.5274

...

782/781 [==============================] - 313s 401ms/step - loss: 0.6884 - acc: 0.7602 - val_loss: 1.1739 - val_acc: 0.6740
Epoch 17/20

Epoch 00017: LearningRateScheduler reducing learning rate to 0.0001.
167/781 [=====>........................] - ETA: 3:53 - loss: 0.6381 - acc: 0.7761

782/781 [==============================] - 313s 401ms/step - loss: 0.6117 - acc: 0.7865 - val_loss: 0.6528 - val_acc: 0.7840
Epoch 18/20

Epoch 00018: LearningRateScheduler reducing learning rate to 0.0001.
173/781 [=====>........................] - ETA: 3:51 - loss: 0.6009 - acc: 0.7903

782/781 [==============================] - 314s 401ms/step - loss: 0.5932 - acc: 0.7946 - val_loss: 0.6721 - val_acc: 0.7807
Epoch 19/20

Epoch 00019: LearningRateScheduler reducing learning rate to 0.0001.
175/781 [=====>........................] - ETA: 3:50 - loss: 0.5906 - acc: 0.7963

782/781 [==============================] - 313s 401ms/step - loss: 0.5786 - acc: 0.7983 - val_loss: 0.6190 - val_acc: 0.7935
Epoch 20/20

Epoch 00020: LearningRateScheduler reducing learning rate to 0.0001.
175/781 [=====>........................] - ETA: 3:50 - loss: 0.5732 - acc: 0.7978

782/781 [==============================] - 313s 400ms/step - loss: 0.5768 - acc: 0.7981 - val_loss: 0.6633 - val_acc: 0.7807

Epoch 00020: saving model to cifar10_weights.20-0.663.hdf5


## **For 20 epochs run on downsized images (16x16) our model has learnt good weights and given us training accuracy of 79.81% and validation accuracy of 78.07%**

In [0]:
# Save the trained weights in to .h5 format
pre_model28_l.save_weights("pre_model28_l_weights.h5")
print("Saved model to disk")

from google.colab import files

files.download('pre_model28_l_weights.h5')

Saved model to disk


**Just checking out weights at a particular layer**

In [0]:
len(pre_model28_l.layers[2].get_weights())

4

In [0]:
pre_model28_l.layers[2].get_weights()[0]

array([0.8468731 , 0.6476582 , 0.7165509 , 1.3465536 , 0.57304126,
       0.8646428 , 1.1702656 , 0.8359851 , 0.8980804 , 1.0968549 ,
       0.9116674 , 0.9466966 , 0.7896094 , 0.8556654 , 0.8701283 ,
       0.60723805, 0.56897074, 0.6901295 , 0.9055469 , 1.0864186 ,
       1.4888134 , 1.265754  , 0.7794853 , 0.7476797 , 0.8843203 ,
       1.1372865 , 0.92509574, 0.76956964, 0.9714205 , 0.73980266,
       0.90889096, 1.1885608 ], dtype=float32)

#**TRAINING MODEL ON ORIGINAL IMAGES**

In [0]:
print(num_filter)
print(l)
print(batch_size)


32
20
64


### **Since we are changing the input size back to 32x32 for final training we need to change the output layer Dense connections and so we define "our_output_layer"**

In [0]:
def our_output_layer(input):
    global compression
    BatchNorm = BatchNormalization(name='Batch_output_layer')(input)
    relu = Activation('relu',name='Activation_output_layer')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2),name='AvgPooling_output_layer')(relu)
    flat = Flatten(name='Flattern_output_layer')(AvgPooling)
    output = Dense(num_classes, activation='softmax',name='our_Dense_output_layer')(flat)  #Name of Dense layer is different than output_layer's Dense layer
    
    return output

# **model28_l is defined** 
### for original images 32x32

In [0]:
#Everthing same except Dense layer name
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3] 
dropout_rate = 0.1

input = Input(shape=(img_height, img_width, channel,),name='input_layer')
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same',name='Conv2D_1st')(input)

block=1
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

block=2
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

block=3
Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

block=4
Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = our_output_layer(Last_Block)       #our_output_layer:- dense layer with different name

  
model28_l = Model(inputs=[input], outputs=[output])

In [0]:
model28_l.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
Conv2D_1st (Conv2D)             (None, 32, 32, 32)   864         input_layer[0][0]                
__________________________________________________________________________________________________
Batch10 (BatchNormalization)    (None, 32, 32, 32)   128         Conv2D_1st[0][0]                 
__________________________________________________________________________________________________
Activation10 (Activation)       (None, 32, 32, 32)   0           Batch10[0][0]                    
__________________________________________________________________________________________________
Conv10 (Co

Concat42 (Concatenate)          (None, 4, 4, 64)     0           Concat41[0][0]                   
                                                                 Dropout42[0][0]                  
__________________________________________________________________________________________________
Batch43 (BatchNormalization)    (None, 4, 4, 64)     256         Concat42[0][0]                   
__________________________________________________________________________________________________
Activation43 (Activation)       (None, 4, 4, 64)     0           Batch43[0][0]                    
__________________________________________________________________________________________________
Conv43 (Conv2D)                 (None, 4, 4, 16)     9216        Activation43[0][0]               
__________________________________________________________________________________________________
Dropout43 (Dropout)             (None, 4, 4, 16)     0           Conv43[0][0]                     
__________

In [0]:
! pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id': '1-BtbHl6BE6p1mJ8DvkvwUfOE1so9ZeJ3'})
downloaded.GetContentFile('pre_model28_l_weights.h5')

## **Now we load weights learnt from pre_model28_l into our new model28_l except for the last layer (output layer of densenet).**
by_name=True will help for that purpose

In [0]:
model28_l.load_weights('pre_model28_l_weights.h5',by_name=True)  # Load weights for all layers except Dense layer

In [0]:
model28_l.layers[2].get_weights()[0]

array([0.8468731 , 0.6476582 , 0.7165509 , 1.3465536 , 0.57304126,
       0.8646428 , 1.1702656 , 0.8359851 , 0.8980804 , 1.0968549 ,
       0.9116674 , 0.9466966 , 0.7896094 , 0.8556654 , 0.8701283 ,
       0.60723805, 0.56897074, 0.6901295 , 0.9055469 , 1.0864186 ,
       1.4888134 , 1.265754  , 0.7794853 , 0.7476797 , 0.8843203 ,
       1.1372865 , 0.92509574, 0.76956964, 0.9714205 , 0.73980266,
       0.90889096, 1.1885608 ], dtype=float32)

## **Note the definition of lr_changes_1 which uses lr_schedule_1 to change the learning rate as we have defined. Now model28_l is compiled**

In [0]:
# determine Loss function and Optimizer
our_weight_checksaver = keras.callbacks.ModelCheckpoint('our_cifar10_weights.{epoch:02d}-{val_loss:.3f}.hdf5',
                                                  monitor='val_loss', verbose=1, save_weights_only=True, period=3, mode = 'auto')

lr_changes_1 = keras.callbacks.LearningRateScheduler(lr_schedule_1, verbose=1)

#define optimizer
opt_adam = keras.optimizers.Adam(lr=0.001,decay=1e-8)

model28_l.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['accuracy'])

## **Final run for 30 epochs**

In [0]:
epochs=30
batch_size=64
our_history=model28_l.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),
                      steps_per_epoch=len(x_train)/batch_size,
                      epochs=epochs, 
                      verbose=1,
                      callbacks=[our_weight_checksaver,lr_changes_1],
                      validation_data=(x_test,y_test)
                     )

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
619/781 [======================>.......] - ETA: 2:21 - loss: 0.6842 - acc: 0.7666

782/781 [==============================] - 724s 926ms/step - loss: 0.6632 - acc: 0.7733 - val_loss: 1.1940 - val_acc: 0.6922
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
302/781 [==========>...................] - ETA: 6:38 - loss: 0.5504 - acc: 0.8096

782/781 [==============================] - 698s 892ms/step - loss: 0.5371 - acc: 0.8156 - val_loss: 0.6171 - val_acc: 0.8074
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
212/781 [=======>......................] - ETA: 7:52 - loss: 0.4880 - acc: 0.8342

782/781 [==============================] - 697s 892ms/step - loss: 0.4923 - acc: 0.8304 - val_loss: 0.5908 - val_acc: 0.8185

Epoch 00003: saving model to our_cifar10_weights.03-0.591.hdf5
Epoch 4/30

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
152/781 [====>.........................] - ETA: 8:43 - loss: 0.4560 - acc: 0.8414

782/781 [==============================] - 697s 891ms/step - loss: 0.4636 - acc: 0.8396 - val_loss: 0.8003 - val_acc: 0.7760
Epoch 5/30

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
170/781 [=====>........................] - ETA: 8:29 - loss: 0.4536 - acc: 0.8451

...

782/781 [==============================] - 698s 893ms/step - loss: 0.1865 - acc: 0.9362 - val_loss: 0.3529 - val_acc: 0.9041
Epoch 27/30

Epoch 00027: LearningRateScheduler reducing learning rate to 0.0002.
172/781 [=====>........................] - ETA: 8:26 - loss: 0.1778 - acc: 0.9368

782/781 [==============================] - 698s 893ms/step - loss: 0.1861 - acc: 0.9345 - val_loss: 0.3578 - val_acc: 0.8997

Epoch 00027: saving model to our_cifar10_weights.27-0.358.hdf5
Epoch 28/30

Epoch 00028: LearningRateScheduler reducing learning rate to 0.0001.
142/781 [====>.........................] - ETA: 8:52 - loss: 0.1734 - acc: 0.9400

782/781 [==============================] - 699s 893ms/step - loss: 0.1772 - acc: 0.9384 - val_loss: 0.3224 - val_acc: 0.9093
Epoch 29/30

Epoch 00029: LearningRateScheduler reducing learning rate to 0.0001.
166/781 [=====>........................] - ETA: 8:31 - loss: 0.1615 - acc: 0.9419

782/781 [==============================] - 697s 891ms/step - loss: 0.1690 - acc: 0.9398 - val_loss: 0.3067 - val_acc: 0.9110
Epoch 30/30

Epoch 00030: LearningRateScheduler reducing learning rate to 0.0001.
172/781 [=====>........................] - ETA: 8:27 - loss: 0.1731 - acc: 0.9370

782/781 [==============================] - 698s 892ms/step - loss: 0.1723 - acc: 0.9391 - val_loss: 0.3216 - val_acc: 0.9098

Epoch 00030: saving model to our_cifar10_weights.30-0.322.hdf5


### **Training accuracy: 93.91% and validation accuracy: 90.98% at the end of 30th epoch**


In [0]:
# Test the model
score = model28_l.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 55s 6ms/step
Test loss: 0.32159562615156173
Test accuracy: 0.9098


In [0]:
# Save the trained weights in to .h5 format
model28_l.save_weights("model28_l_weights_final.h5")
print("Saved weights to disk")
model28_l.save('model28_l.h5')
print("Saved model to disk")




Saved weights to disk
Saved model to disk


In [0]:
from google.colab import files
files.download('model28_l_weights_final.h5')
files.download('model28_l.h5')



In [0]:
model28_l_weights = model28_l.get_weights()
np.save("model28_l_final_weights_array", model28_l_weights)
files.download('model28_l_final_weights_array.npy')